In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import pytorch_lightning
from nlp_datasets import get_dataloader

import torch
from torch import optim
from torch.optim.lr_scheduler import StepLR

import gc

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
torch.cuda.empty_cache()
gc.collect()

654

In [3]:
text_decoder_id = f"MLP-KTLim/llama-3-Korean-Bllossom-8B"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
tokenizer = AutoTokenizer.from_pretrained(
    text_decoder_id,
    lagacy = False
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
base_model = AutoModelForCausalLM.from_pretrained(
    text_decoder_id,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True, 
    quantization_config=bnb_config)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
for name, module in base_model.named_modules():
    print(name)


model
model.embed_tokens
model.layers
model.layers.0
model.layers.0.self_attn
model.layers.0.self_attn.q_proj
model.layers.0.self_attn.k_proj
model.layers.0.self_attn.v_proj
model.layers.0.self_attn.o_proj
model.layers.0.self_attn.rotary_emb
model.layers.0.mlp
model.layers.0.mlp.gate_proj
model.layers.0.mlp.up_proj
model.layers.0.mlp.down_proj
model.layers.0.mlp.act_fn
model.layers.0.input_layernorm
model.layers.0.post_attention_layernorm
model.layers.1
model.layers.1.self_attn
model.layers.1.self_attn.q_proj
model.layers.1.self_attn.k_proj
model.layers.1.self_attn.v_proj
model.layers.1.self_attn.o_proj
model.layers.1.self_attn.rotary_emb
model.layers.1.mlp
model.layers.1.mlp.gate_proj
model.layers.1.mlp.up_proj
model.layers.1.mlp.down_proj
model.layers.1.mlp.act_fn
model.layers.1.input_layernorm
model.layers.1.post_attention_layernorm
model.layers.2
model.layers.2.self_attn
model.layers.2.self_attn.q_proj
model.layers.2.self_attn.k_proj
model.layers.2.self_attn.v_proj
model.layers.2.

In [6]:
train_dataloader = get_dataloader()
valid_dataloader = get_dataloader(split='valid')

dataset_dict:  {'prompt': '\n        <|user|>\n        title : [보수약정금]〈북한주민인 피고들과 위임 및 보수약정을 체결한 법무법인인 원고가 위임약정에 따른 업무 수행 후 피고들을 상대로 성공보수금을 청구한 사건〉 [공2024상,727]\n【판시사항】\n[1] 북한주민이 상속 등으로 취득한 재산 그 자체는 물론이고 그 재산을 처분한 대가로 얻은 재산을 처분하는 행위 역시 남북 주민 사이의 가족관계와 상속 등에 관한 특례법 제15조에 따라 재산관리인을 통하지 않은 경우, 무효인지 여부(적극) 및 그러한 법률행위가 재산관리인이 선임되기 전에 있었다고 하더라도 마찬가지인지 여부(적극)\n[2] 일부무효의 법리를 정한 민법 제137조에서 ‘당사자의 의사’의 의미 / 여러 개의 계약 전부가 경제적, 사실적으로 일체로서 행하여져서 하나의 계약인 것과 같은 관계에 있는 경우, 법률행위의 일부무효 법리가 적용되는지 여부(적극)\n[3] 변호사에게 계쟁 사건의 처리를 위임하는 경우, 보수 지급 및 수액에 관하여 명시적인 약정을 아니하였더라도 보수를 지급할 묵시의 약정이 있는 것으로 보아야 하는지 여부(원칙적 적극) 및 이 경우 보수액을 결정하는 방법\n[4] 갑 법무법인이 남북 주민 사이의 가족관계와 상속 등에 관한 특례법의 적용을 받는 북한주민인 을 등과 을 등의 병에 대한 친생자확인소송, 상속회복 청구소송 등에 관한 위임약정 및 ‘총상속지분의 30% 또는 이에 상응하는 금전’을 성공보수로 받는 보수약정을 체결하였고, 이후 병의 상속인들 사이에 화해가 성립하여 을 등이 상속재산을 분할받자, 갑 법무법인이 을 등을 상대로 위 보수약정에 따라 상속재산 30%에 해당하는 돈의 지급을 구하였으나, 을 등이 위 보수약정은 재산관리인을 통하지 아니하고 상속재산 등에 관하여 한 법률행위이므로, 위 특례법 제15조에 따라 무효라고 주장한 사안에서, 제반 사정에 비추어 위 보수약정이 무효로 된다고 하더라도 양 당사자는 위 위임계약을 체결, 

Map:   0%|          | 0/764 [00:00<?, ? examples/s]

dataset_dict:  {'prompt': '\n        <|user|>\n        title : [성폭력범죄의처벌등에관한특례법위반(카메라등이용촬영ㆍ반포등)ㆍ아동ㆍ청소년의성보호에관한법률위반(성착취물제작ㆍ배포등)ㆍ아동ㆍ청소년의성보호에관한법률위반(성착취물소지등)ㆍ정보통신망이용촉진및정보보호등에관한법률위반ㆍ(음란물유포)]〈「아동·청소년의 성보호에 관한 법률」 제11조 제3항의 아동·청소년성착취물 ‘배포’ 및 같은 조 제5항의 아동·청소년성착취물 ‘소지’ 여부가 문제된 사건〉 [공2023하,2040]\n【판시사항】\n[1] 아동·청소년의 성보호에 관한 법률 제11조 제3항에서 정한 아동·청소년성착취물의 ‘배포’ 및 ‘공연히 전시’하는 행위의 의미 / 자신의 웹사이트에 아동·청소년성착취물이 저장된 다른 웹사이트로 연결되는 링크를 게시하여 불특정 또는 다수인이 링크를 이용하여 별다른 제한 없이 아동·청소년성착취물에 바로 접할 수 있는 상태를 실제로 조성한 경우, 위 조항에서 정한 아동·청소년성착취물을 배포하거나 공연히 전시한다는 구성요건을 충족하는지 여부(적극)\n[2] 아동·청소년의 성보호에 관한 법률 제11조 제5항에서 정한 아동·청소년성착취물 ‘소지’의 의미 및 피고인이 자신이 지배하지 않는 서버 등에 저장된 아동·청소년성착취물에 접근하였으나 위 성착취물을 다운로드하는 등 실제로 지배할 수 있는 상태로 나아가지는 않은 경우, 이를 아동·청소년성착취물을 ‘소지’한 것으로 평가할 수 있는지 여부(원칙적 소극)\n【판결요지】\n[1] 아동·청소년의 성보호에 관한 법률 제11조 제3항은 “아동·청소년성착취물을 배포·제공하거나 이를 목적으로 광고·소개하거나 공연히 전시 또는 상영한 자는 3년 이상의 징역에 처한다.”라고 규정하고 있다. 여기서 아동·청소년성착취물의 ‘배포’란 아동·청소년성착취물을 불특정 또는 다수인에게 교부하는 것을 의미하고, ‘공연히 전시’하는 행위란 불특정 또는 다수인이 실제로 아동·청소년성착취물을 인식할 수 있는 상태에 두는 것을 의미한다.

Map:   0%|          | 0/85 [00:00<?, ? examples/s]

In [7]:
## ---  LoRA --- ##
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    modules_to_save=["classifier"],
)
model = get_peft_model(base_model, config)
model.print_trainable_parameters()

peft_model_id = f"MLP-KTLim/llama-3-Korean-Bllossom-8B_Lora"

trainable params: 4,718,592 || all params: 8,172,867,584 || trainable%: 0.0577


In [8]:
num_epochs = 10
learning_rate = 2e-5
optimizer = optim.Adam(
    model.parameters(),
    lr=learning_rate,
)
lr_scheduler = StepLR(optimizer, step_size=1, gamma=0.85)

In [9]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [11]:
from tqdm import tqdm

device = "cuda"
model = model.to(device)
scaler = torch.cuda.amp.GradScaler()

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        # print(batch)
        with torch.autocast(device_type='cuda'):
            outputs = model(**batch)
            loss = outputs.loss
            total_loss += loss.detach().float()
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        # loss.backward()
        # optimizer.step()
        # lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    eval_loss = 0
    eval_preds = []
    for step, batch in enumerate(tqdm(valid_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.detach().float()
        eval_preds.extend(
            tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), skip_special_tokens=True)
        )

    eval_epoch_loss = eval_loss / len(valid_dataloader)
    eval_ppl = torch.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(train_dataloader)
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

  0%|          | 0/764 [00:00<?, ?it/s]

  0%|          | 0/764 [00:00<?, ?it/s]


RuntimeError: User specified an unsupported autocast device_type 'gpu'

In [ ]:
model.save_pretrained(peft_model_id)